# Create Agent with Function Definition

In this notebook we will create an Agent for Amazon Bedrock using the new capabilities for function definition.

We will use an HR agent as example. With this agent, you can check your available vacation days and request a new vacation leave. We will use an AWS Lambda function to define the logic that checks for the number of available vacation days and confirm new time off.

For this example, we will manage employee data in an [SQLite](https://www.sqlite.org/) database and generate synthetic data for demonstrating the agent.

## Prerequisites
Before starting, let's update the botocore and boto3 packages to ensure we have the latest version

In [ ]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli
!pip install --upgrade -q opensearch-py

Let's now check the boto3 version to ensure the correct version has been installed. Your version should be greater than or equal to 1.34.90.

In [ ]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
import time
import os
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
print(boto3.__version__)

In [ ]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Let's now create the boto3 clients for the required AWS services

In [ ]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
open_search_serverless_client = boto3.client('opensearchserverless')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
bedrock_client = boto3.client('bedrock')

Next we can set some configuration variables for the agent and for the lambda function being created

In [ ]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

In [ ]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "city-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for providing citizen assistance at AnyCity"
agent_instruction = "You are an helpful agent, helping citizens at AnyCity to create park reservation, and check on garbage pickup dates.  You also have access to documentation about the city in your knowledge base.  Use the documentation to help answer questions."
agent_action_group_name = "AnyCityActionGroup"
agent_action_group_description = "Actions for getting the park reservations, creating park reservations, and checking garbage pick up times"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

## Creating Lambda Function

We will now create a lambda function that interacts with the SQLite file `employee_database.db`. To do so we will:
1. Create the `employee_database.db` file which contains the employee database with some generated data.
2. Create the `lambda_function.py` file which contains the logic for our lambda function
3. Create the IAM role for our Lambda function
4. Create the lambda function infrastructure with the required permissions

In [ ]:
import sqlite3
import random
from datetime import date, timedelta

# Connect to the SQLite database (creates a new one if it doesn't exist)
conn = sqlite3.connect('anycity_database.db')
c = conn.cursor()

# Create the citizen table
c.execute('''CREATE TABLE IF NOT EXISTS citizen
             (citizen_id INTEGER PRIMARY KEY AUTOINCREMENT,
              citizen_name TEXT,
              citizen_address TEXT,
              district INTEGER)''')

# Create the garbage table
c.execute('''CREATE TABLE IF NOT EXISTS garbage
             (district_id INTEGER PRIMARY KEY,
              pickup_day TEXT)''')

# Create the park table
c.execute('''CREATE TABLE IF NOT EXISTS park_reservation
             (reservation_id INTEGER PRIMARY KEY AUTOINCREMENT,
              citizen_id INTEGER,
              park_id INTEGER,
              reservation_start_date TEXT,
              reservation_end_date TEXT,
              FOREIGN KEY(citizen_id) REFERENCES citizen(citizen_id))''')

# Generate some random data for 50 citizens
citizen_first_name = ['John', 'Jane', 'Bob', 'Alice', 'Tom','Emily', 'Michael', 'Sarah', 'David', 'Jessica']
citizen_last_name = ['Doe', 'Smith', 'Johnson', 'Williams', 'Brown', 'Davis', 'Wilson', 'Taylor', 'Anderson', 'Thompson']
streets = ['Main St', 'Oak Ave', 'Maple Rd', 'Cedar Ln', 'Pine Blvd']

for i in range(50):
    name = f"{random.choice(citizen_first_name)} {random.choice(citizen_last_name)}"
    address = f"{random.randint(100, 999)} {random.choice(streets)}"
    district = random.randint(1, 5)
    c.execute("INSERT INTO citizen (citizen_name, citizen_address, district) VALUES (?, ?, ?)",
              (name, address, district))

# Generate garbage pickup schedule for each district
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
for district in range(1, 6):
    pickup_day = random.choice(days)
    c.execute("INSERT INTO garbage (district_id, pickup_day) VALUES (?, ?)",
              (district, pickup_day))

# Generate some random park reservations
num_parks = 3
for _ in range(20):
    citizen_id = random.randint(1, 50)
    park_id = random.randint(1, num_parks)
    start_date = date.today() + timedelta(days=random.randint(1, 30))
    end_date = start_date + timedelta(days=random.randint(1, 3))
    c.execute("INSERT INTO park_reservation (citizen_id, park_id, reservation_start_date, reservation_end_date) VALUES (?, ?, ?, ?)",
              (citizen_id, park_id, start_date.isoformat(), end_date.isoformat()))

# Commit the changes and close the connection
conn.commit()
conn.close()

Let's now create our lambda function. It implements the functionality for `get_available_park_days` for a given park_id and `book_park` for an citizen giving a start and end date

In [ ]:
%%writefile lambda_function.py
import os
import json
import shutil
import sqlite3
from datetime import datetime, timedelta

def get_available_park_days(park_id, start_date, end_date):
    conn = sqlite3.connect('/tmp/anycity_database.db')
    c = conn.cursor()

    try:
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        
        c.execute("""
            SELECT reservation_start_date, reservation_end_date 
            FROM park_reservation 
            WHERE park_id = ? AND 
                  (reservation_start_date <= ? AND reservation_end_date >= ?)
        """, (park_id, end_date.strftime('%Y-%m-%d'), start_date.strftime('%Y-%m-%d')))
        
        reservations = c.fetchall()
        
        all_days = set(start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1))
        
        for reservation in reservations:
            res_start = datetime.strptime(reservation[0], '%Y-%m-%d')
            res_end = datetime.strptime(reservation[1], '%Y-%m-%d')
            reserved_days = set(res_start + timedelta(days=x) for x in range((res_end - res_start).days + 1))
            all_days -= reserved_days
        
        available_days = sorted(list(all_days))
        
        conn.close()
        return [day.strftime('%Y-%m-%d') for day in available_days]
    except Exception as e:
        conn.close()
        raise Exception(f"Error occurred: {e}")

def book_park(citizen_id, park_id, start_date, end_date):
    conn = sqlite3.connect('/tmp/anycity_database.db')
    c = conn.cursor()

    try:
        c.execute("SELECT * FROM citizen WHERE citizen_id = ?", (citizen_id,))
        if not c.fetchone():
            conn.close()
            return f"Citizen with ID {citizen_id} does not exist."

        available_days = get_available_park_days(park_id, start_date, end_date)
        required_days = set(datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=x) for x in range((datetime.strptime(end_date, '%Y-%m-%d') - datetime.strptime(start_date, '%Y-%m-%d')).days + 1))
        
        if not all(day.strftime('%Y-%m-%d') in available_days for day in required_days):
            conn.close()
            return "The selected dates are not fully available for reservation."

        c.execute("""
            INSERT INTO park_reservation (citizen_id, park_id, reservation_start_date, reservation_end_date) 
            VALUES (?, ?, ?, ?)
        """, (citizen_id, park_id, start_date, end_date))

        conn.commit()
        conn.close()
        return f"Park reservation successful for citizen ID {citizen_id} from {start_date} to {end_date}."
    except Exception as e:
        conn.rollback()
        conn.close()
        raise Exception(f"Error occurred: {e}")

def get_garbage_pickup_day(district_id):
    conn = sqlite3.connect('/tmp/anycity_database.db')
    c = conn.cursor()

    try:
        c.execute("SELECT pickup_day FROM garbage WHERE district_id = ?", (district_id,))
        result = c.fetchone()
        
        if result:
            pickup_day = result[0]
            conn.close()
            return pickup_day
        else:
            conn.close()
            return f"No garbage pickup information found for district ID {district_id}."
    except Exception as e:
        conn.close()
        raise Exception(f"Error occurred: {e}")

def lambda_handler(event, context):
    original_db_file = 'anycity_database.db'
    target_db_file = '/tmp/anycity_database.db'
    if not os.path.exists(target_db_file):
        shutil.copy2(original_db_file, target_db_file)
        
    # Retrieve agent session attributes for context 
    session_attributes = event.get('sessionAttributes', {})
    first_name = session_attributes.get('firstName', '')
    citizen_id = session_attributes.get('citizenID', '2')
    district_id = session_attributes.get('districtID', '4')
    
    
    agent = event['agent']
    actionGroup = event['actionGroup']
    function = event['function']
    parameters = event.get('parameters', [])
    responseBody = {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }

    if function == 'get_available_park_days':
        park_id = None
        start_date = None
        end_date = None
        for param in parameters:
            if param["name"] == "park_id":
                park_id = param["value"]
            if param["name"] == "start_date":
                start_date = param["value"]
            if param["name"] == "end_date":
                end_date = param["value"]

        if not all([park_id, start_date, end_date]):
            raise Exception("Missing mandatory parameters: park_id, start_date, end_date")
        
        available_days = get_available_park_days(park_id, start_date, end_date)
        responseBody = {
            'TEXT': {
                "body": f"Available days for park ID {park_id} from {start_date} to {end_date}: {available_days}"
            }
        }
    elif function == 'book_park':
        #citizen_id = None
        park_id = None
        start_date = None
        end_date = None
        for param in parameters:
            # if param["name"] == "citizen_id":
            #     citizen_id = param["value"]
            if param["name"] == "park_id":
                park_id = param["value"]
            if param["name"] == "start_date":
                start_date = param["value"]
            if param["name"] == "end_date":
                end_date = param["value"]
            
        if not all([citizen_id, park_id, start_date, end_date]):
            raise Exception("Missing mandatory parameters: citizen_id, park_id, start_date, end_date")
        
        completion_message = book_park(citizen_id, park_id, start_date, end_date)
        responseBody = {
            'TEXT': {
                "body": completion_message
            }
        }
    elif function == 'get_garbage_pickup_day':
        # district_id = None
        # for param in parameters:
        #     if param["name"] == "district_id":
        #         district_id = param["value"]
        
        if not district_id:
            raise Exception("Missing mandatory parameter: district_id")
        
        pickup_day = get_garbage_pickup_day(district_id)
        responseBody = {
            'TEXT': {
                "body": f"Garbage pickup day for district ID {district_id}: {pickup_day}"
            }
        }

    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

Next let's create the lambda IAM role and policy to invoke a Bedrock model

In [ ]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

We can now package the lambda function to a Zip file and create the lambda function using boto3

In [ ]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.write("anycity_database.db")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

# Create Knowledge Base

In [ ]:
bucket_name = 'reinvent2024-anycity-kb'
s3_client.create_bucket(Bucket=bucket_name)

In [ ]:
kb_files_path = 'data'
kb_key = 'kb_documents'
# Upload Knowledge Base files to this s3 bucket
for f in os.listdir(kb_files_path):
    if f.endswith(".pdf"):
        s3_client.upload_file(kb_files_path+'/'+f, bucket_name, kb_key+'/'+f)

In [ ]:
# Select Amazon titan as the embedding model
embedding_model_arn = f'arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v2:0'

In [ ]:
kb_bedrock_allow_policy_name = f"bd-kb-bedrock-allow-{suffix}"

# Create IAM policies for KB to invoke embedding model
bedrock_kb_allow_fm_model_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                embedding_model_arn
            ]
        }
    ]
}

kb_bedrock_policy_json = json.dumps(bedrock_kb_allow_fm_model_policy_statement)

kb_bedrock_policy = iam_client.create_policy(
    PolicyName=kb_bedrock_allow_policy_name,
    PolicyDocument=kb_bedrock_policy_json
)


In [ ]:
kb_aoss_allow_policy_name = f"bd-kb-aoss-allow-{suffix}"

# Create IAM policies for KB to access OpenSearch Serverless
bedrock_kb_allow_aoss_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "aoss:APIAccessAll",
            "Resource": [
                f"arn:aws:aoss:{region}:{account_id}:collection/*"
            ]
        }
    ]
}


kb_aoss_policy_json = json.dumps(bedrock_kb_allow_aoss_policy_statement)

kb_aoss_policy = iam_client.create_policy(
    PolicyName=kb_aoss_allow_policy_name,
    PolicyDocument=kb_aoss_policy_json
)

In [ ]:
kb_s3_allow_policy_name = f"bd-kb-s3-allow-{suffix}"

kb_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowKBAccessDocuments",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}/*",
                f"arn:aws:s3:::{bucket_name}"
            ],
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": f"{account_id}"
                }
            }
        }
    ]
}


kb_s3_json = json.dumps(kb_s3_allow_policy_statement)
kb_s3_policy = iam_client.create_policy(
    PolicyName=kb_s3_allow_policy_name,
    PolicyDocument=kb_s3_json
)

In [ ]:
kb_role_name = f'AmazonBedrockExecutionRoleForKnowledgeBase_bedrock_docs'

# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
kb_role = iam_client.create_role(
    RoleName=kb_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_aoss_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_s3_policy['Policy']['Arn']
)

In [ ]:
kb_role_arn = kb_role["Role"]["Arn"]
kb_role_arn

## Create Vector Database

First let's create a vector store. In this section we will use Amazon OpenSerach Serverless.

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.

In [ ]:
kb_collection_name = f'bd-kbc-{suffix}'

# Create OpenSearch Collection
security_policy_json = {
    "Rules": [
        {
            "ResourceType": "collection",
            "Resource":[
                f"collection/{kb_collection_name}"
            ]
        }
    ],
    "AWSOwnedKey": True
}
security_policy = open_search_serverless_client.create_security_policy(
    description='security policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(security_policy_json),
    type='encryption'
)


In [ ]:
network_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "dashboard"
      },
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "collection"
      }
    ],
    "AllowFromPublic": True
  }
]

network_policy = open_search_serverless_client.create_security_policy(
    description='network policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(network_policy_json),
    type='network'
)


In [ ]:
response = sts_client.get_caller_identity()
current_role = response['Arn']
current_role

In [ ]:
data_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "Permission": [
          "aoss:DescribeCollectionItems",
          "aoss:CreateCollectionItems",
          "aoss:UpdateCollectionItems",
          "aoss:DeleteCollectionItems"
        ],
        "ResourceType": "collection"
      },
      {
        "Resource": [
          f"index/{kb_collection_name}/*"
        ],
        "Permission": [
            "aoss:CreateIndex",
            "aoss:DeleteIndex",
            "aoss:UpdateIndex",
            "aoss:DescribeIndex",
            "aoss:ReadDocument",
            "aoss:WriteDocument"
        ],
        "ResourceType": "index"
      }
    ],
    "Principal": [
        kb_role_arn,
        f"arn:aws:sts::{account_id}:assumed-role/Admin/*",
        current_role
    ],
    "Description": ""
  }
]

data_policy = open_search_serverless_client.create_access_policy(
    description='data access policy for aoss collection',
    name=kb_collection_name,
    policy=json.dumps(data_policy_json),
    type='data'
)


In [ ]:
opensearch_collection_response = open_search_serverless_client.create_collection(
    description='OpenSearch collection for Amazon Bedrock Knowledge Base',
    name=kb_collection_name,
    standbyReplicas='DISABLED',
    type='VECTORSEARCH'
)
opensearch_collection_response

In [ ]:
collection_arn = opensearch_collection_response["createCollectionDetail"]["arn"]
collection_arn

In [ ]:
# wait for collection creation
response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    time.sleep(30)
    response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
print('\nCollection successfully created:')
print(response["collectionDetails"])
# Extract the collection endpoint from the response
host = (response['collectionDetails'][0]['collectionEndpoint'])
final_host = host.replace("https://", "")
final_host

## Create OpenSearch Index

Let's now create a vector index to index our data

In [ ]:
kb_vector_index_name = "bedrock-knowledge-base-index"

# Set up AWS authentication
service = 'aoss'
credentials = boto3.Session().get_credentials()
awsauth = AWSV4SignerAuth(credentials, region, service)

# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': final_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

# It can take up to a minute for data access rules to be enforced
time.sleep(45)

# Define index settings and mappings
index_settings = {
    "settings": {
        "index.knn": "true"
    },
    "mappings": {
        "properties": {
            "vector": {
                "type": "knn_vector",
                "dimension": 1024,
                 "method": {
                     "name": "hnsw",
                     "engine": "faiss",
                     "space_type": "innerproduct",
                     "parameters": {
                         "ef_construction": 512,
                         "m": 16
                     },
                 },
             },
            "text": {
                "type": "text"
            },
            "text-metadata": {
                "type": "text"
            }
        }
    }
}

# Create index
response = oss_client.indices.create(index=kb_vector_index_name, body=json.dumps(index_settings))
print(response)

## Create Knowledge Base

In [ ]:
# kb_metadataField = 'bedrock-knowledge-base-metadata'
# kb_textField = 'bedrock-knowledge-base-text'
# kb_vectorField = 'bedrock-knowledge-base-vector'

storageConfiguration={
        'opensearchServerlessConfiguration': {
            'collectionArn': collection_arn,
            'fieldMapping': {
                'metadataField': 'text-metadata',
                'textField': 'text',
                'vectorField': 'vector'
            },
            'vectorIndexName': kb_vector_index_name
        },
        'type': 'OPENSEARCH_SERVERLESS'
    }




In [ ]:
kb_name = f'bedrockdocs-kb-{region}-{account_id}'

# Creating the knowledge base
try:
    # ensure the index is created and available
    time.sleep(45)
    kb_obj = bedrock_agent_client.create_knowledge_base(
        name=kb_name, 
        description='KB that contains the bedrock documentation',
        roleArn=kb_role_arn,
        knowledgeBaseConfiguration={
            'type': 'VECTOR',  # Corrected type
            'vectorKnowledgeBaseConfiguration': {
                'embeddingModelArn': embedding_model_arn
            }
        },
        storageConfiguration=storageConfiguration
    )

    # Pretty print the response
    pprint.pprint(kb_obj)

except Exception as e:
    print(f"Error occurred: {e}")

In [ ]:
data_source_name = f'bedrock-docs-kb-docs-{suffix}'

# Define the S3 configuration for your data source
s3_configuration = {
    'bucketArn': f"arn:aws:s3:::{bucket_name}",
    'inclusionPrefixes': [kb_key]
}

# Define the data source configuration
data_source_configuration = {
    's3Configuration': s3_configuration,
    'type': 'S3'
}

knowledge_base_id = kb_obj["knowledgeBase"]["knowledgeBaseId"]
knowledge_base_arn = kb_obj["knowledgeBase"]["knowledgeBaseArn"]

chunking_strategy_configuration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# Create the data source
try:
    # ensure that the KB is created and available
    time.sleep(45)
    data_source_response = bedrock_agent_client.create_data_source(
        knowledgeBaseId=knowledge_base_id,
        name=data_source_name,
        description='DataSource for the bedrock documentation',
        dataSourceConfiguration=data_source_configuration,
        vectorIngestionConfiguration = {
            "chunkingConfiguration": chunking_strategy_configuration
        }
    )

    # Pretty print the response
    pprint.pprint(data_source_response)

except Exception as e:
    print(f"Error occurred: {e}")

## Start Ingestion Job

Once the Knowledge Base and Data Source are created, we can start the ingestion job. During the ingestion job, Knowledge Base will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case Amazon OpenSource Serverless.

In [ ]:
# Start an ingestion job
data_source_id = data_source_response["dataSource"]["dataSourceId"]
start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=knowledge_base_id,
    dataSourceId=data_source_id
)

## Create Agent
We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation for a specific foundation model and the agent IAM role with the policy associated to it. 

In [ ]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [ ]:
bedrock_agent_kb_allow_policy_name = f"bda-kb-allow-{suffix}"

bedrock_agent_kb_retrival_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve"
            ],
            "Resource": [
                knowledge_base_arn
            ]
        }
    ]
}
bedrock_agent_kb_json = json.dumps(bedrock_agent_kb_retrival_policy_statement)
agent_kb_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_kb_allow_policy_name,
    Description=f"Policy to allow agent to retrieve documents from knowledge base.",
    PolicyDocument=bedrock_agent_kb_json
)

In [ ]:
agent_role_name

In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_kb_schema_policy['Policy']['Arn']
)

### Creating the agent
Once the needed IAM role is created, we can use the Bedrock Agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. Note that the agent created is not yet prepared. Later, we will prepare and use the agent.

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

Let's now store the agent id in a local variable to use it on subsequent steps.

In [ ]:
agent_id = response['agent']['agentId']
agent_id

## Create Agent Action Group
We will now create an agent action group that uses the lambda function created earlier. The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group capabilities, we provide an action group description.

In this example, we provide the Action Group functionality using a `functionSchema`. You can alternatively provide an `APISchema`. The notebook [02-create-agent-with-api-schema.ipynb](02-create-agent-with-api-schema/02-create-agent-with-api-schema.ipynb) provides an example of that approach.

To define the functions using a function schema, you need to provide the `name`, `description` and `parameters` for each function.

In [ ]:
agent_functions = [
    {
        'name': 'get_available_park_days',
        'description': 'Get the available days for a specific park within a date range.  If no end data is specified assume the end_date is the same as the start_date',
        'parameters': {
            "park_id": {
                "description": "The ID of the park to check availability",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "The start date of the date range to check (format: YYYY-MM-DD)",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "The end date of the date range to check (format: YYYY-MM-DD)",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'book_park',
        'description': 'Book a park for a specific citizen within a date range.  If no end data is specified assume the end_date is the same as the start_date',
        'parameters': {
            # "citizen_id": {
            #     "description": "The ID of the citizen making the reservation",
            #     "required": True,
            #     "type": "integer"
            # },
            "park_id": {
                "description": "The ID of the park to be reserved",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "The start date of the reservation (format: YYYY-MM-DD)",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "The end date of the reservation (format: YYYY-MM-DD)",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'get_garbage_pickup_day',
        'description': 'Get the garbage pickup day',
        'parameters': {
            # "district_id": {
            #     "description": "The ID of the district to get the garbage pickup day",
            #     "required": True,
            #     "type": "integer"
            # }
        }
    }
]

In [ ]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [ ]:
agent_action_group_response

## Allowing Agent to invoke Action Group Lambda
Before using the action group, we need to allow the agent to invoke the lambda function associated with the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [ ]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


response

## Add Knowledge Base to the Agent

In [ ]:
agent_kb_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {kb_name} knowledge base to provide accurate responses to the questions about AnyCity USA.',
    knowledgeBaseId=knowledge_base_id
)

## Preparing Agent

Let's create a DRAFT version of the agent that can be used for internal testing.


In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

In [ ]:
agent_alias_name = "PKALIASID"

# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
# Pause to make sure agent alias is ready
time.sleep(30)

In [ ]:
response = bedrock_client.create_guardrail(
    name='AnyCityGuardrail',
    description='Guardrail for city assitant to help users with questions about AnyCity',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Politics',
                'definition': 'Guardrail to prevent from engaging in political discussions, expressing political opinions, or taking partisan stances on contested issues.',
                'examples': [
                    'Who do you think should win the upcoming election?',
                    'Can you explain why candidate X would be better than candidate Y?',
                    'Do you think current gun regulations are too strict or too lenient?',
                    'Help me write a convincing argument on the Freedom Party platform and their key policies.',
                    'Who is tougher on crime?  Republicans or Democrats?',
                ],
                'type': 'DENY'
            },
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'republican'},
            {'text': 'democrat'},
            {'text': 'conservative'},
            {'text': 'liberal'},
            {'text': 'immigration'},
            {'text': 'abortion'},
            {'text': 'bipartisan'},
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
    },
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.7
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.7
            }
        ]
    },
    blockedInputMessaging='Sorry, your question violates our usage policies. Please contact the AnyCity City Administrator at 987-654-3210 if this message was generated in error.',
    blockedOutputsMessaging='Sorry, I am unable to reply. Please contact the AnyCity City Administrator at 987-654-3210',
)

## Adding the guardrail to your agent

Now that we have a Bedrock guardrail we need to integrate it with your agent. So let's associate our guardrail with the agent now

In [ ]:
guardrail_id = response['guardrailId']
guardrail_version = response['version']
bedrock_agent_client.update_agent(
                agentId=agent_id,
                agentName=agent_name,
                foundationModel=agent_foundation_model,
                guardrailConfiguration = {
                    'guardrailIdentifier': guardrail_id,
                    'guardrailVersion': guardrail_version
                },
                agentResourceRoleArn=agent_role['Role']['Arn']
            )
time.sleep(30)

In [ ]:
old_alias_id = agent_alias_id

response = bedrock_agent_client.create_agent_alias(
    agentAliasName='AgentWithGuardrail',
    agentId=agent_id,
    description='Test alias with Guardrails for Amazon Bedrock',
)

alias_id = response["agentAlias"]["agentAliasId"]

print("The Agent alias is:",alias_id)
time.sleep(30)

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

## Invoke Agent

Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [ ]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

In [ ]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
#    inputText="How much is bulk item pickup?",
    inputText="What day is trash pickup?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session,
    sessionState={
        "sessionAttributes": {
            "firstName": "Aaron",
            "citizenID": "2",
            "districtID": "3"
        }
    }
)

logger.info(pprint.pprint(agentResponse))

In [ ]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [ ]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

In [ ]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="What day is trash pickup?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session,
    sessionState={
        "sessionAttributes": {
            "firstName": "Aaron",
            "citizenID": "2",
            "districtID": "3"
        }
    }
)

logger.info(pprint.pprint(agentResponse))

In [ ]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [ ]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="now let me reserve park 1 from May 15 2025 to Jun 15 2025.",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session,
    sessionState={
        "sessionAttributes": {
            "firstName": "Aaron",
            "citizenID": "2",
            "districtID": "3"
        }
    }
)

logger.info(pprint.pprint(agentResponse))

In [ ]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [ ]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [ ]:
simple_agent_invoke("What day does trash get picked up for district 3?", agent_id, agent_alias_id, session_id)

In [ ]:
simple_agent_invoke("Who can I contact about a large trash pickup?", agent_id, agent_alias_id, session_id)

In [ ]:
simple_agent_invoke("Is park 2 available on Feb 31st?", agent_id, agent_alias_id, session_id, enable_trace=True)

# Guardrails

In [ ]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="I'd like the reserve the park for an immigration party on April 19 2025 for citizen 12",
    agentId=agent_id,
    agentAliasId=alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

In [ ]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

## Clean up (optional)

The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:

1. update the action group to disable it
2. delete agent action group
4. delete agent
5. delete lambda function
6. delete the created IAM roles and policies


In [ ]:
import time

def wait_for_deletion(resource_type, identifier, max_attempts=10):
    for _ in range(max_attempts):
        if not resource_exists(resource_type, identifier):
            return True
        time.sleep(10)
    return False

try:
    bedrock_agent_client.delete_agent_alias(
        agentId=agent_id,
        agentAliasId=agent_alias['agentAlias']['agentAliasId']
    )
    print(f"Deleted agent alias: {agent_alias['agentAlias']['agentAliasId']}")
    # Wait for alias deletion to complete
    wait_for_deletion('agent_alias', agent_alias['agentAlias']['agentAliasId'])
except Exception as e:
    print(f"Error deleting agent alias: {e}")

try:
    action_groups = bedrock_agent_client.list_agent_action_groups(agentId=agent_id, agentVersion='DRAFT')
    for action_group in action_groups['agentActionGroupSummaries']:
        action_group_id = action_group['actionGroupId']
        action_group_name = action_group['actionGroupName']
        bedrock_agent_client.update_agent_action_group(
            agentId=agent_id,
            agentVersion='DRAFT',
            actionGroupId=action_group_id,
            actionGroupName=action_group_name,
            actionGroupState='DISABLED',
        )
        bedrock_agent_client.delete_agent_action_group(
            agentId=agent_id,
            agentVersion='DRAFT',
            actionGroupId=action_group_id
        )
        print(f"Deleted agent action group: {action_group_id}")
except Exception as e:
    print(f"Error deleting agent action groups: {e}")

try:
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    bucket.objects.all().delete()
    bucket.delete()
    print(f"Deleted S3 bucket: {bucket_name}")
except Exception as e:
    print(f"Error deleting S3 bucket: {e}")

try:
    bedrock_agent_client.delete_agent(agentId=agent_id)
    print(f"Deleted agent: {agent_id}")
    wait_for_deletion('agent', agent_id)
except Exception as e:
    print(f"Error deleting agent: {e}")
    print("Attempting to force delete the agent...")
    try:
        bedrock_agent_client.delete_agent(agentId=agent_id, skipResourceInUseCheck=True)
        print(f"Force deleted agent: {agent_id}")
        wait_for_deletion('agent', agent_id)
    except Exception as e2:
        print(f"Error force deleting agent: {e2}")

try:
    lambda_client.delete_function(FunctionName=lambda_function_name)
    print(f"Deleted Lambda function: {lambda_function_name}")
except Exception as e:
    print(f"Error deleting Lambda function: {e}")

def detach_and_delete_policies(role_name):
    try:
        role = iam_client.get_role(RoleName=role_name)
        attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']
        
        for policy in attached_policies:
            iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy['PolicyArn'])
            if not policy['PolicyArn'].startswith('arn:aws:iam::aws:policy/'):  # Don't delete AWS managed policies
                iam_client.delete_policy(PolicyArn=policy['PolicyArn'])
        
        inline_policies = iam_client.list_role_policies(RoleName=role_name)['PolicyNames']
        for policy_name in inline_policies:
            iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
        
        iam_client.delete_role(RoleName=role_name)
        print(f"Deleted role and its policies: {role_name}")
    except Exception as e:
        print(f"Error deleting role {role_name}: {e}")

for role_name in [agent_role_name, kb_role_name, lambda_function_role]:
    detach_and_delete_policies(role_name)

try:
    open_search_serverless_client.delete_collection(
        id=opensearch_collection_response["createCollectionDetail"]["id"]
    )
    print(f"Deleted OpenSearch collection: {kb_collection_name}")
    wait_for_deletion('opensearch', kb_collection_name)

    for policy_type in ['network', 'encryption']:
        try:
            open_search_serverless_client.delete_security_policy(
                name=kb_collection_name,
                type=policy_type
            )
            print(f"Deleted OpenSearch {policy_type} policy: {kb_collection_name}")
        except Exception as e:
            print(f"Error deleting OpenSearch {policy_type} policy: {e}")

    try:
        open_search_serverless_client.delete_access_policy(
            name=kb_collection_name,
            type='data'
        )
        print(f"Deleted OpenSearch data access policy: {kb_collection_name}")
    except Exception as e:
        print(f"Error deleting OpenSearch data access policy: {e}")

except Exception as e:
    print(f"Error deleting OpenSearch resources: {e}")
    
try:
    bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=knowledge_base_id)
    print(f"Deleted Knowledge Base: {knowledge_base_id}")
    wait_for_deletion('knowledge_base', knowledge_base_id)
except Exception as e:
    print(f"Error deleting Knowledge Base: {e}")

    
try:
    bedrock_client.delete_guardrail(guardrailIdentifier=guardrail_id)
    print(f"Deleted Guardrail: {guardrail_id}")
except Exception as e:
    print(f"Error deleting Guardrail: {e}")

try:
    data_sources = bedrock_agent_client.list_data_sources(knowledgeBaseId=knowledge_base_id)
    for data_source in data_sources['dataSourceSummaries']:
        bedrock_agent_client.delete_data_source(
            knowledgeBaseId=knowledge_base_id,
            dataSourceId=data_source['dataSourceId']
        )
        print(f"Deleted data source: {data_source['dataSourceId']}")
except Exception as e:
    print(f"Error deleting data sources: {e}")

try:
    logs_client = boto3.client('logs')
    log_group_name = f"/aws/lambda/{lambda_function_name}"
    logs_client.delete_log_group(logGroupName=log_group_name)
    print(f"Deleted CloudWatch log group: {log_group_name}")
except Exception as e:
    print(f"Error deleting CloudWatch logs: {e}")

def resource_exists(resource_type, identifier):
    try:
        if resource_type == 'agent':
            bedrock_agent_client.get_agent(agentId=identifier)
        elif resource_type == 'lambda':
            lambda_client.get_function(FunctionName=identifier)
        elif resource_type == 's3':
            s3_client.head_bucket(Bucket=identifier)
        elif resource_type == 'opensearch':
            open_search_serverless_client.batch_get_collection(names=[identifier])
        elif resource_type == 'knowledge_base':
            bedrock_agent_client.get_knowledge_base(knowledgeBaseId=identifier)
        elif resource_type == 'guardrail':
            bedrock_client.get_guardrail(guardrailId=identifier)
        return True
    except:
        return False

resources_to_check = [
    ('agent', agent_id),
    ('lambda', lambda_function_name),
    ('s3', bucket_name),
    ('opensearch', kb_collection_name),
    ('knowledge_base', knowledge_base_id),
    ('guardrail', guardrail_id)
]

for resource_type, identifier in resources_to_check:
    if resource_exists(resource_type, identifier):
        print(f"Warning: {resource_type} {identifier} still exists")
    else:
        print(f"{resource_type.capitalize()} {identifier} has been successfully deleted")

# Check for IAM roles
for role_name in [agent_role_name, kb_role_name, lambda_function_role]:
    try:
        iam_client.get_role(RoleName=role_name)
        print(f"Warning: IAM role {role_name} still exists")
    except iam_client.exceptions.NoSuchEntityException:
        print(f"IAM role {role_name} has been successfully deleted")

## Conclusion
We have now experimented with using boto3 SDK to create, invoke and delete an agent created using function definitions.

## Take aways
Adapt this notebook to create new agents using function definitions for your application

## Thank You